<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Perhaps the simplest flow problem is that of a graph of only two vertices
with an edge from source (0) to sink (1)
```

(0) --5--> (1)

```
Here, the maximum flow is simply the capacity of the edge:


In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
graph = csr_matrix([[0, 5], [0, 0]])
maximum_flow(graph, 0, 1).flow_value

5

In [ ]:
maximum_flow(graph, 0, 1, method='edmonds_karp').flow_value

5

If, on the other hand, there is a bottleneck between source and sink, that
can reduce the maximum flow
```

(0) --5--> (1) --3--> (2)

```

In [ ]:
graph = csr_matrix([[0, 5, 0], [0, 0, 3], [0, 0, 0]])
maximum_flow(graph, 0, 2).flow_value

3

A less trivial example is given in [2], Chapter 26.1:


In [ ]:
graph = csr_matrix([[0, 16, 13,  0,  0,  0],
                    [0,  0, 10, 12,  0,  0],
                    [0,  4,  0,  0, 14,  0],
                    [0,  0,  9,  0,  0, 20],
                    [0,  0,  0,  7,  0,  4],
                    [0,  0,  0,  0,  0,  0]])
maximum_flow(graph, 0, 5).flow_value

23

It is possible to reduce the problem of finding a maximum matching in a
bipartite graph to a maximum flow problem: Let $G = ((U, V), E)$ be a
bipartite graph. Then, add to the graph a source vertex with edges to every
vertex in $U$ and a sink vertex with edges from every vertex in
$V$. Finally, give every edge in the resulting graph a capacity of 1.
Then, a maximum flow in the new graph gives a maximum matching in the
original graph consisting of the edges in $E$ whose flow is positive.

Assume that the edges are represented by a
$\lvert U \rvert \times \lvert V \rvert$ matrix in CSR format whose
$(i, j)$'th entry is 1 if there is an edge from $i \in U$ to
$j \in V$ and 0 otherwise; that is, the input is of the form required
by :func:`maximum_bipartite_matching`. Then the CSR representation of the
graph constructed above contains this matrix as a block. Here's an example:


In [ ]:
graph = csr_matrix([[0, 1, 0, 1], [1, 0, 1, 0], [0, 1, 1, 0]])
print(graph.toarray())

[[0 1 0 1]
 [1 0 1 0]
 [0 1 1 0]]

In [ ]:
i, j = graph.shape
n = graph.nnz
indptr = np.concatenate([[0],
                         graph.indptr + i,
                         np.arange(n + i + 1, n + i + j + 1),
                         [n + i + j]])
indices = np.concatenate([np.arange(1, i + 1),
                          graph.indices + i + 1,
                          np.repeat(i + j + 1, j)])
data = np.ones(n + i + j, dtype=int)

graph_flow = csr_matrix((data, indices, indptr))
print(graph_flow.toarray())

[[0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0]
 [0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]]

At this point, we can find the maximum flow between the added sink and the
added source and the desired matching can be obtained by restricting the
flow function to the block corresponding to the original graph:


In [ ]:
result = maximum_flow(graph_flow, 0, i+j+1, method='dinic')
matching = result.flow[1:i+1, i+1:i+j+1]
print(matching.toarray())

[[0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]]

This tells us that the first, second, and third vertex in $U$ are
matched with the second, first, and third vertex in $V$ respectively.

While this solves the maximum bipartite matching problem in general, note
that algorithms specialized to that problem, such as
:func:`maximum_bipartite_matching`, will generally perform better.

This approach can also be used to solve various common generalizations of
the maximum bipartite matching problem. If, for instance, some vertices can
be matched with more than one other vertex, this may be handled by
modifying the capacities of the new graph appropriately.